In [1]:
# Lab 6 Softmax Classifier
import tensorflow as tf
import numpy as np
tf.set_random_seed(777)  # for reproducibility

In [47]:
# Predicting animal type based on various features
xy = np.loadtxt('data-04-zoo.csv', delimiter=',', dtype=np.float32)
x_data = xy[:, 0:-1]
y_data = xy[:, [-1]]

print(x_data.shape, y_data.shape)
x_data

(101, 16) (101, 1)


array([[ 1.,  0.,  0., ...,  0.,  0.,  1.],
       [ 1.,  0.,  0., ...,  1.,  0.,  1.],
       [ 0.,  0.,  1., ...,  1.,  0.,  0.],
       ..., 
       [ 1.,  0.,  0., ...,  1.,  0.,  1.],
       [ 0.,  0.,  1., ...,  0.,  0.,  0.],
       [ 0.,  1.,  1., ...,  1.,  0.,  0.]], dtype=float32)

In [48]:
nb_classes = 7  # 0 ~ 6

X = tf.placeholder(tf.float32, [None, 16])
Y = tf.placeholder(tf.int32, [None, 1])  # 0 ~ 6
Y_one_hot = tf.one_hot(Y, nb_classes)  # one hot
print("one_hot", Y_one_hot)
Y_one_hot = tf.reshape(Y_one_hot, [-1, nb_classes])
print("reshape", Y_one_hot)

one_hot Tensor("one_hot_3:0", shape=(?, 1, 7), dtype=float32)
reshape Tensor("Reshape_15:0", shape=(?, 7), dtype=float32)


In [49]:
W = tf.Variable(tf.random_normal([16, nb_classes]), name='weight')
b = tf.Variable(tf.random_normal([nb_classes]), name='bias')

# tf.nn.softmax computes softmax activations
# softmax = exp(logits) / reduce_sum(exp(logits), dim)
logits = tf.matmul(X, W) + b
hypothesis = tf.nn.softmax(logits)

# Cross entropy cost/loss
cost_i = tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=Y_one_hot)
cost = tf.reduce_mean(cost_i)
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.1).minimize(cost)

In [50]:
prediction = tf.argmax(hypothesis, 1)
correct_prediction = tf.equal(prediction, tf.argmax(Y_one_hot, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [51]:
# Launch graph
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())

    for step in range(2000):
        sess.run(optimizer, feed_dict={X: x_data, Y: y_data})
        if step % 500 == 0:
            loss, acc = sess.run([cost, accuracy], feed_dict={X: x_data, Y: y_data})
            print("Step: {:5}\tLoss: {:.3f}\tAccuracy: {:.2%}".format(step, loss, acc))
    
    pred = sess.run(prediction, feed_dict={X: x_data})

Step:     0	Loss: 5.430	Accuracy: 18.81%
Step:   500	Loss: 0.205	Accuracy: 94.06%
Step:  1000	Loss: 0.110	Accuracy: 99.01%
Step:  1500	Loss: 0.073	Accuracy: 100.00%


In [56]:
y_flat = np.squeeze(y_data)
ind = 15
print("[{}] Prediction: {} True Y: {}".format(pred[ind] == int(y_flat[ind]), pred[ind], int(y_flat[ind])))

[True] Prediction: 6 True Y: 6
